In [1]:
from dotenv import load_dotenv
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, HarmBlockThreshold, HarmCategory
from linkedin_lookup_agent import lookup as linkedin_lookup_agent
from linkedin import scrape_linkedin_profile 
import os
import requests

load_dotenv()

True

In [2]:
linkedin_profile_url = linkedin_lookup_agent(name="Jason Clishe")

summary_template = """
Given the information {information} about a person, I want you to create:
1. A short summary
2. Two interesting facts about them
"""

summary_prompt_template = PromptTemplate(
    input_variables=["information"],
    template=summary_template
)



> Entering new AgentExecutor chain...
Action: Crawl Google for LinkedIn profile page
Action Input: Jason Clishe["I am a 25+ year computer industry veteran, and have been working with cloud solutions… | Learn more about Jason Clishe's work experience, education, ...", 'Jason Clishe is a freelance photographer near Cincinnati, OH. He specializes in motocross, drone, and glamour photography.', 'Technology Specialist by trade. Ironman triathlete turned cyclist, private pilot, photographer. Member of the Flying Neutrons flying club based at the ...', 'Technology Specialist, private pilot, Ironman triathlete, photographer. Fueled by coffee and craft beer. Cincinnati, OH Joined December 2007.', 'Read writing from Jason Clishe on Medium. Cloud Solution Architect at CDW, GCP Professional Cloud Architect, 2-time Ironman triathlon finisher, ...', "I'm happy to share that I've obtained a new certification: Google Cloud Professional Cloud DevOps Engineer from Google!", 'Professionally, I help org

In [3]:
print(linkedin_profile_url)

I am sorry, I could not find a LinkedIn profile for Jason Clishe.


In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    temperature=0.8,
    streaming=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [ ]:
chain = LLMChain(llm=llm, prompt=summary_prompt_template)

In [ ]:
linkedin_data = scrape_linkedin_profile(
    linkedin_profile_url=linkedin_profile_url
    )

In [ ]:
print(chain.run(information=linkedin_data))